# **Machine Learning for Factor Investing**

This notebook implements and expands on the ideas presented in [Coqueret and Guida (2021)](http://www.mlfactor.com/), please refer to their book as it provides a really good, data-oriented introduction to quantitative equity investing. My goal in this exercise is for nothing more than my own practice and learning, and all credit goes to the original authors.

I recommend running this notebook on [Google Colab](https://colab.research.google.com/), where you can up and running in only a few seconds. You might just need to download the [original data](https://github.com/shokru/mlfactor.github.io/blob/master/material/data_ml.RData) to your Google Drive.

### **Chapter 0.** Importing stuff and setup

In [ ]:
# Basic stuff
import pandas as pd
import numpy as np
import datetime
from calendar import monthrange
from dateutil.relativedelta import relativedelta

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn")
plt.rcParams[
    "patch.facecolor"
] = "white"  # This is helpful if you're using Colab in dark mode
plt.rcParams["figure.figsize"] = 15, 7
# import graphviz

# Reading Rdata - need to install in Collab
# !pip install pyreadr
import pyreadr

# Regression type stuff
from statsmodels.formula.api import ols
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.linear_model import Lasso, Ridge, ElasticNet

from portfolio import *

# # Trees
# from sklearn import tree
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
# # from xgboost import XGBClassifier

# # Deep learning stuff
# import tensorflow as tf
# from tensorflow import keras

### **Chapter 1.** Notations and data

Mostly ingesting the data and some quick and dirty data cleaning as well as initial exploration.

In [ ]:
universe = pd.read_csv("../data/ibov_universe.csv", index_col=None)


universe = Portifolio(universe)
data = universe.pre_processing()
numeric_columns = universe.numeric_columns

In [ ]:
# bigest returns:
data[["ticker", "ret12m"]].sort_values(by="ret12m", ascending=False).head(30)
data = data.drop(index=[1151, 963, 964, 990, 579, 883, 868, 274])
# data.loc[data['ticker'] == 'GGBR4',['date','ret12m','fator_cotacao','mkt_value','size','closed_price']]
# if fator_cotacao changes than display index:

In [ ]:
data["size"] = (
    data.groupby("date")["mkt_value"]
    .apply(lambda x: (x > x.median()))
    .reset_index(drop=True)
    .replace({True: "Large", False: "Small"})
)
data["year"] = data["date"].dt.year

return_by_size = data.groupby(["year", "size"])["ret12m"].mean().reset_index()

ax = sns.barplot(x="year", y="ret12m", hue="size", data=return_by_size)
ax.set(xlabel="", ylabel="Average return")
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

In [ ]:
plt.figure(figsize=(15, 6))  # definindo o tamanho da figura
df = data.groupby(["year"])["ticker"].size().reset_index(name="count")

sns.barplot(data=df, x="year", y="count", width=0.8)
plt.xlabel("ano")
plt.ylabel("Número de tickers")
plt.show()

In [ ]:
data.head()

Visualização da quantidade de ativos por data, podemos verificar que o ibov cresceu em número de ações ordinárias ao longo do tempo.

In [ ]:
data.groupby("date")["ticker"].count().plot(
    ylabel="n_assets", title="Number of distinct assets through time"
);

Vamos normalizar as colunas numéricas:

In [ ]:
numeric_columns

In [ ]:
# normalize numerical columns:
# import minmax scaler
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
transformer = scaler.fit(data[numeric_columns])
data[numeric_columns] = transformer.transform(data[numeric_columns])

# removendo elementos que estejam fora de 3 desvios padrões
data = data[(np.abs(data[numeric_columns]) < 3).all(axis=1)]

verificando normalização:

In [ ]:
data.loc[(data["date"] == "2022-12-29"), "mkt_value"].hist(bins=100);

In [ ]:
features = data.columns.to_list()[3:-7]
features_short = [
    "ebit12m",
    "ativo_total",
    "net_worth",
    "liq_corr",
    "vol12m",
    "mkt_value",
    "entreprise_value",
]

In [ ]:
features

#### Separação de amostra de treino e teste

In [ ]:
# find separation_date:
separation_date = data["date"].unique()[int(len(data["date"].unique()) * 0.8)]
print(f"separation_date: {separation_date}")

In [ ]:
separation_mask = data["date"] < separation_date

training_sample = data.loc[separation_mask]
testing_sample = data.loc[~separation_mask]

In [ ]:
stock_ids = data["ticker"].unique().tolist()

max_dates = data.groupby("ticker")["date"].count().max()
stocks_with_max_dates = data.groupby("ticker")["date"].count() == max_dates
stock_ids_short = (
    stocks_with_max_dates.where(stocks_with_max_dates).dropna().index.tolist()
)  # these are stocks who have data for all timestamps

returns = data[data["ticker"].isin(stock_ids_short)][["date", "ticker", "ret1m"]]
returns = returns.pivot(index="date", columns="ticker");

## Factor Investing


Vamos agrupar os dados do nosso universo e realizar a segmentação das ações por tamanho, no

In [ ]:
data["mkt_value"].describe()
# data.groupby('date')['VM'].apply(lambda x: (x > x.median()))

In [ ]:
data["size"] = (
    data.groupby("date")["mkt_value"]
    .apply(lambda x: (x > x.median()))
    .reset_index(drop=True)
    .replace({True: "Large", False: "Small"})
)
data["year"] = data["date"].dt.year

return_by_size = data.groupby(["year", "size"])["ret12m"].mean().reset_index()

ax = sns.barplot(x="year", y="ret12m", hue="size", data=return_by_size)
ax.set(xlabel="", ylabel="Average return")
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

In [ ]:
data[["ticker", "ret12m", "date"]].sort_values(by="ret12m", ascending=False).head(10)

In [ ]:
# lendo os fatores:
ff_factors = pd.read_csv("../data/risk_factors/factors.csv", index_col=None)
# ajustando os tipos das colunas:
ff_factors["date"] = pd.to_datetime(ff_factors["date"])
# ff_factors['date'] = ff_factors['date'].dt.strftime("%Y/%m/%d")
ff_factors.rename({"Risk_free": "RF", "Rm_minus_Rf": "MKT_RF"}, axis=1, inplace=True)
columns_to_float = ff_factors.columns[1:]
ff_factors[columns_to_float] = ff_factors[columns_to_float].astype(float)
display(ff_factors.head())

In [ ]:
temp_factors = ff_factors.copy()

temp_factors["date"] = temp_factors["date"].dt.year
temp_factors = pd.melt(temp_factors, id_vars="date")
temp_factors = temp_factors.groupby(["date", "variable"]).mean().reset_index()

plt = sns.lineplot(x="date", y="value", hue="variable", data=temp_factors)
plt.legend(bbox_to_anchor=(1.05, 0.7), loc=2, borderaxespad=0.0)
plt.set_title("Average returns over time of common factors");
# replicating this from the book for completeness only, but i think it's a pretty messy chart
# it's hard to take much insight from it

In [ ]:
# let's see how factors cumulative performance over time
# but wrap that in a function that allows you to choose the start period (as that influences cumulative performance a lot)


def plot_cumulative_performance(df, start_date=None):
    # this function will plot cumulative performance for any wide dataframe of returns (e.g. index is date, columns are assets/factor)
    # optional: you can pass the start date in %m/%d/%y format e.g. '1/1/1995', '12/15/2000'
    # if you don't pass a start date, it will use the whole sample

    cumul_returns = (1 + df.set_index("date")).cumprod()

    if start_date is None:
        start_date = cumul_returns.index.min()
    else:
        start_date = datetime.strptime(start_date, "%m/%d/%Y")
        cumul_returns = cumul_returns.loc[cumul_returns.index >= start_date]

    first_line = pd.DataFrame(
        [[1.0 for col in cumul_returns.columns]],
        columns=cumul_returns.columns,
        index=[start_date - relativedelta(months=1)],
    )

    cumul_returns = pd.concat([first_line, cumul_returns])

    return cumul_returns.plot(
        title=f'Cumulative factor performance since {start_date.strftime("%B %Y")}'
    )


plot_cumulative_performance(ff_factors)

Below we perform **Fama-Macbeth regressions**, which is the standard way of validating a factor's risk premium in the cross-section of stock returns. As we will see, this involves a two-step process of:

*   Time-series regression: regress each asset's returns on factors, i.e. one regression per asset. Store the coefficients.
*   Cross-section regression: regress each asset's returns on coefficients obtained in previous step, i.e. one regression per time period.


In [ ]:
# merging and cleaning up the data before we run the regressions
data_fm = data[["date", "ticker", "ret1m"]][data["ticker"].isin(stock_ids_short)]
data_fm = data_fm.merge(ff_factors, on="date")
data_fm["ret1m"] = data_fm.groupby("ticker")["ret1m"].shift(1)
data_fm.dropna(inplace=True)

# running time series regressions

reg_output = {}

for k, g in data_fm.groupby("ticker"):
    model = ols("ret1m ~ MKT_RF + SMB + HML + WML + IML", data=g)
    results = model.fit()

    reg_output[k] = results.params

betas = pd.DataFrame.from_dict(reg_output).T
betas.head()

In [ ]:
betas.describe()

In [ ]:
# prepping coeficient data to run second round of regressions
loadings = betas.drop("Intercept", axis=1).reset_index(drop=True)
ret = returns.T.reset_index(drop=True)
fm_data = pd.concat([loadings, ret], axis=1)
fm_data = pd.melt(fm_data, id_vars=["MKT_RF", "SMB", "HML", "WML", "IML"])

# running cross section regressions

reg_output_2 = {}

for k, g in fm_data.groupby("variable"):
    model = ols("value ~ MKT_RF + SMB + HML + WML + IML", data=g)
    results = model.fit()

    reg_output_2[k] = results.params

# refer to the mlfactor book or the fama-macbeth literature for more info on what the gammas stand for
# but you can think of them as an estimate of a given factor's risk premium at a point in time
gammas = (
    pd.DataFrame.from_dict(reg_output_2)
    .T.reset_index()
    .rename({"index": "date"}, axis=1)
)
gammas.head()

In [ ]:
fm_data.head()

In [ ]:
# since we get one estimate of that risk premium for each time step, we can plot how it evolves over time
x = pd.melt(gammas.drop("Intercept", axis=1), id_vars="date")

g = sns.FacetGrid(x, col="variable")
g.map(sns.lineplot, "date", "value")

Below we deploy the factor competition strategy outlined in the book. The main idea here is to regress a factor on the remaining factors and test whether the coefficient is significant. A significant coefficient means that the factors on the right-hand side don't completely explain the factor on the left-hand side, which naturally means the latter is useful.


In [ ]:
factor_comp = pd.melt(ff_factors.drop("RF", axis=1), id_vars="date")

factor_comp = factor_comp.merge(ff_factors.drop("RF", axis=1), on="date")

factor_comp_coefs = {}
factor_comp_tstats = {}

for k, g in factor_comp.groupby("variable"):
    reg_data = g.drop([k, "date", "variable"], axis=1)
    formula = "value ~ " + " + ".join(reg_data.columns.values[1:].tolist())

    model = ols(formula, data=reg_data)
    results = model.fit()

    factor_comp_coefs[k] = results.params
    factor_comp_tstats[k] = results.tvalues

alphas = pd.DataFrame.from_dict(factor_comp_coefs).T
alphas_tstats = pd.DataFrame.from_dict(factor_comp_tstats).T

alphas_tstats

In [ ]:
alphas_table = alphas.round(3).applymap(str)

prob99 = 2.58
prob95 = 1.96

alphas_table[alphas_tstats >= prob99] = alphas_table[alphas_tstats >= prob99] + " (**)"
alphas_table[alphas_tstats.apply(lambda x: x.between(prob95, prob99))] = (
    alphas_table[alphas_tstats.apply(lambda x: x.between(prob95, prob99))] + " (*)"
)

factors = factor_comp.columns[3:].tolist()

alphas_table = alphas_table[["Intercept"] + factors].reindex(factors)

alphas_table

Below we explore factor time series momentum by looking at the partial auto-correlation functions. The shaded lines are confidence intervals, which the *statsmodels* library handily calculates for us by default (at 5% confidence levels).


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf

nfactors = len(factors)

# Plot y from -0.1 to 0.1:
fig, axs = plt.subplots(ncols=nfactors, figsize=(20, 2))

for i, factor in enumerate(factors):
    plot_pacf(ff_factors[factor], ax=axs[i], lags=20, zero=False, title=factor)
    axs[i].set_ylim(-0.1, 0.1)

plt.show()